# 해외뉴스번역

## 키워드 분석
#### 준비하기

In [550]:
import pandas as pd
import numpy as np
import matplotlib as mpl #그래프
import re #특수문자 제거

#### 샘플 데이터 가져오기
> 10112017의 cnn기사

In [551]:
art_sample = pd.read_csv('./articles/Nov2017/csvdataset/10112017/cnn.csv')
art_sample.head(3)

,Unnamed: 0,KEYWORDS,SUMMARY,TEXT,TITLE,URL
0,0,"['despair', 'rides', 'vegetables', 'york', 'we...",The opinions expressed in this commentary are ...,Story highlights Kenneth Podziba: Cycling offe...,Why we'll never stop biking,https://www.cnn.com/2017/11/01/opinions/terror...
1,1,"['david', 'obamas', 'sasha', 'obama', 'axelrod...",Photos: Presidential inaugurations since 1789 ...,David Axelrod is CNN's senior political commen...,David Axelrod: Obama's legacy can't be erased,https://www.cnn.com/2017/01/20/opinions/david-...
2,2,"['temperatures', 'weekend', 'night', 'degrees'...",The National Weather Service had issued a wind...,Story highlights Wind advisory issued for Mass...,"Cold, windy weather to blast Northeast this we...",https://www.cnn.com/2017/11/09/us/cold-weather...


### 키워드를 무슨 근거로 뽑은 건지 모르겠다 - 출처 분석
> sample data에서 첫번째 기사 사용

#### keyword 분석을 위한 keyword data 미리보기

In [552]:
key_data = art_sample['KEYWORDS'][0]
key_data
#type(key_data) ##pandas의 Series형태

"['despair', 'rides', 'vegetables', 'york', 'wells', 'ride', 'riding', 'really', 'bicycle', 'bike', 'stop', 'biking']"

#### 샘플 데이터 전처리
##### 특수문자 제거하기
>##### 특수문자 제거를 위해 사용한 ftn
- `re.sub('[제거할 문자]','대체',data)`
- `list.replace('바꿀문자','대체')`
>##### 단어 단위로 쪼개기
- `list.split()`

In [553]:
keyword = re.sub("['',-.]",'',key_data) ##쓸데없는 문자 제거
keyword = keyword.replace('[','').replace(']','').split(' ')

summary_sp = re.sub('[.\():,""-/]',' ',art_sample['SUMMARY'][0])
summary_sp = summary_sp.replace('\n','').split(' ')

title_sp = art_sample['TITLE'][0].split(' ')

text_sp = re.sub('[.\():,""-/]',' ',art_sample['TEXT'][0])
text_sp = text_sp.replace('\n','').split(' ')

##### 대소문자 구별 없애기
- 모두 소문자로 전처리    `list.lower()`

In [554]:
summary_sp = list(map(str.lower,summary_sp))
title_sp = list(map(str.lower,title_sp))
text_sp = list(map(str.lower,text_sp))

##### Data 정리하기
- 샘플 데이터에서 keyword로 분류된 단어가 각각 summary, title, text에 등장하는지의 여부를 논리연산자로 반환

In [555]:
sample_0 = pd.DataFrame({'word':keyword})
sample_0 = sample_0.assign(summary = sample_0['word'].isin(summary_sp), ## summary에 등장하는지
               title = sample_0['word'].isin(title_sp), ## title에 등장하는지
               text = sample_0['word'].isin(text_sp)) ## text에 등장하는지
sample_0

,word,summary,title,text
0,despair,True,False,True
1,rides,True,False,True
2,vegetables,False,False,True
3,york,True,False,True
4,wells,True,False,True
5,ride,False,False,True
6,riding,True,False,True
7,really,False,False,True
8,bicycle,True,False,True
9,bike,True,False,True


> **keyword라고 해서 꼭 원문(text)에 등장한 것은 아니다.**
- text에만 등장한 경우
- summary와 text에 등장한 경우
- title에만 등장한 경우

### 특수문자 제거한 TEXT(기사 원문) word 빈도분석

##### 단어 출현 횟수를 저장하기 위한 리스트 생성

In [556]:
text_fre = list()

for i in range(len(text_sp)):
    text_fre.append(text_sp.count(text_sp[i]))

##### 단어와 출현 빈도를 포함한 데이터프레임 생성

In [567]:
text_sample_2 = pd.DataFrame({'word':list(map(str.lower,text_sp)),'fre':text_fre}).drop_duplicates().sort_values('fre',ascending=False)
## 중복제거 ## fre값 기준 내림차순 정렬

text_sample_2['keyword'] = text_sample_2['word'].isin(keyword) ## text의 단어가 keyword 리스트에도 포함되는지 여부

text_sample_2 = text_sample_2[text_sample_2['keyword']==True] ## 적어도 2번 이상 출현한 단어 & keyword 분류에 포함되는 단어

text_sample_2 ## keyword의 단어가 text(원문)에 몇번 등장하는지

,word,fre,keyword
36,bike,4,True
111,despair,4,True
106,bicycle,3,True
49,york,3,True
345,vegetables,2,True
210,ride,2,True
316,really,2,True
95,wells,2,True
34,riding,2,True
364,rides,2,True


> **keyword라고 해서 text에 자주 등장하지 않는다**
- keyword임에도 text에서 1번 등장한 단어가 존재(bike, riding)
- text에 등장하지 않는(title에만 등장) 경우에는 어떤 기준으로 keyword로 분류하는지?

- 같은 단어를 다른 방식으로 표현하는 경우(rides - ride - riding): 형태는 다르지만 같은 의미
- 비슷한 의미의 다른 형태(bike - bicycle - biking)

**=> 어떻게 취합할 것인지**

### text 빈도분석과 keyword 출처분석 결과 비교하기 (작업중)

In [591]:
#text_sample_3 = text_sample_2.iloc[:,0:2]
#result = pd.concat([sample_0, text_sample_3], axis = 1, join = 'outer',
                 # keys = 'word')
sample_0['word'] == text_sample_2['word']

ValueError: Can only compare identically-labeled Series objects

## 키워드 분석 - 전체 데이터 적용 (작업중)
#### 데이터 전처리 모듈 만들기

In [558]:
def preprocess(path):
    x = pd.read_csv(path)
    #x_shape_0 = x.shape
    #x = x.dropna(axis = 0, how = 'any') # 하나라도 결측치가 있는 row는 삭제
    #print('삭제된 데이터: ',x_shape_0[0] - x.shape[0])
    
    # 초기 array 지정
    keyword = re.sub("[':/\',-.]",'',x['KEYWORDS'][0]).replace('[','').replace(']','').split(' ')
    keyword = list(map(str.lower,keyword))
    summary = re.sub("[':/',-.]","",str(x['SUMMARY'][0])).replace('\n','').split(' ')
    summary = list(map(str.lower,summary))
    title = re.sub("[':/\',-.]",'',x['TITLE'][0]).replace("'","").replace('"','').split(' ')
    title = list(map(str.lower,title))
    text = re.sub("['':/,-.]",'',str(x['TEXT'][0])).replace('\n','').split(' ')
    text = list(map(str.lower,text))
    
    ans_0 = np.array([[keyword,summary,title,text]])
    
    # 나머지 data 추가
    for i in range(1,x.shape[0]):
        keyword = re.sub("[':/\',-.]",'',x['KEYWORDS'][i]).replace('[','').replace(']','').split(' ')
        keyword = list(map(str.lower,keyword))
        summary = re.sub("[':/',-.]","",str(x['SUMMARY'][i])).replace('\n','').split(' ')
        summary = list(map(str.lower,summary))
        title = re.sub("['',:/\-.]",'',x['TITLE'][i]).replace("'","").replace('"','').split(' ')
        title = list(map(str.lower,title))
        text = re.sub("['',-:/.]",'',str(x['TEXT'][i])).replace('\n','').split(' ')
        text = list(map(str.lower,text))
        
        ans_t = np.array([[keyword,summary,title,text]])
        if ans_t.shape == (1,4):
            ans = np.append(ans_0,ans_t,axis = 0)
            ans_0 = ans
        
    print(' 전처리 완료 데이터: ',ans.shape[0],'개','\n','전처리 과정에서 삭제된 데이터: ',x.shape[0] - ans.shape[0],'개')

    return ans

In [559]:
s = preprocess('./articles/Nov2017/Nov2017/csvdataset/11082017/foxnews.csv')

 전처리 완료 데이터:  350 개 
 전처리 과정에서 삭제된 데이터:  1 개


#### keyword 출처를 나타내는 bool형 array 생성하기 (작업중)***여기서부터***

In [564]:
def isin_keyword(x):
    n = x[0].max()
    ans_0 = np.array([[1,4,]])
    print(ans_0.shape,ans_0[[0]])
    
    for i in range(1, x.shape[0]):
        key = pd.Series(x[i,0])
        summ = pd.Series(x[i,1])
        tt = pd.Series(x[i,2])
        txt = pd.Series(x[i,3])
        
        k_summary = key.isin(summ)
        k_title = key.isin(tt)
        k_text = key.isin(txt)
        
        ans_t = np.array([[key,k_summary,k_title,k_text]])
        print(ans_t.shape)
        ans = np.append(ans_0,ans_t,axis = 0)
        ans_0 = ans
    return ans

In [565]:
s_b = isin_keyword(s)

TypeError: 'list' object cannot be interpreted as an integer